<a href="https://colab.research.google.com/github/dorothyro/Power-Platform-App-Dev-in-a-Day-KR/blob/main/competition_baseline_hy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = str(0)
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)
tf.config.experimental.enable_op_determinism()

In [2]:
# 모델 학습을 위한 데이터 다운로드
import kagglehub

path = kagglehub.dataset_download("asaniczka/mammals-image-classification-dataset-45-animals")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mammals-image-classification-dataset-45-animals' dataset.
Path to dataset files: /kaggle/input/mammals-image-classification-dataset-45-animals


In [3]:
# 모델 학습을 위한 데이터 준비
import tensorflow as tf
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split

DATA_DIR = path + "/mammals"

image_paths = []
labels = []

for class_name in sorted(os.listdir(DATA_DIR)):
    class_dir = os.path.join(DATA_DIR, class_name)
    if os.path.isdir(class_dir):
        files = tf.io.gfile.glob(os.path.join(class_dir, "*.jpg"))
        image_paths.extend(files)
        labels.extend([class_name] * len(files))

class_names = sorted(list(set(labels)))
class_to_index = {name: idx for idx, name in enumerate(class_names)}
index_to_class = {idx: name for name, idx in class_to_index.items()}
numeric_labels = [class_to_index[label] for label in labels]

train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths,
    numeric_labels,
    test_size=0.2,
    stratify=numeric_labels,
    shuffle=True,
    random_state=42
)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths,
    temp_labels,
    test_size=0.5,
    stratify=temp_labels,
    shuffle=True,
    random_state=42
)

print(f"Train: {len(train_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}")

def path_label_to_dataset(paths, labels, batch_size=300, img_size=(256,256)):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def load_and_preprocess(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

    ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = path_label_to_dataset(train_paths, train_labels)
val_ds   = path_label_to_dataset(val_paths, val_labels)
test_ds  = path_label_to_dataset(test_paths, test_labels)

Train: 11000, Val: 1375, Test: 1376


In [4]:
# 모델 학습 - 001
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dropout, Dense,
    BatchNormalization, GlobalAveragePooling2D,
    Rescaling,
    Flatten
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers

inputs = Input(shape=(256, 256, 3))
x = Conv2D(32, (3, 3), padding="same", activation='relu', name="conv1")(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
outputs = Dense(45, activation='softmax', name="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[early_stop, lr_scheduler])
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Dense)                 │ (None, 45)             │    23,593,005 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,593,901 (90.00 MB)

 Trainable params: 23,593,901 (90.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 193ms/step - accuracy: 0.0404 - loss: 30.0735 - val_accuracy: 0.0524 - val_loss: 4.0029 - learning_rate: 0.0010
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.0748 - loss: 3.6928 - val_accuracy: 0.1447 - val_loss: 3.3929 - learning_rate: 0.0010
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.3194 - loss: 2.8202 - val_accuracy: 0.2189 - val_loss: 3.0431 - learning_rate: 0.0010
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.6068 - loss: 1.7006 - val_accuracy: 0.2444 - val_loss: 3.0003 - learning_rate: 0.0010
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.8455 - loss: 0.8431 - val_accuracy: 0.2400 - val_loss: 3.2284 - learning_rate: 0.0010
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.9417 - loss: 0.3959 - val_accuracy: 0.2291 - val_loss: 3.4051 - learning_rate: 0.0010
Epoch 7/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.9820 - loss: 0.1773


In [7]:
# baseline - 002 데이터 증강만
import os
import random
import numpy as np
import tensorflow as tf
import kagglehub
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dropout, Dense,
    BatchNormalization, GlobalAveragePooling2D,
    Rescaling,
    Flatten # Flatten 레이어 추가
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers


# Ensure determinism (copied from nnjWuDM8E1Rg for self-contained execution)
os.environ['PYTHONHASHSEED'] = str(0)
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)
tf.config.experimental.enable_op_determinism()

# 모델 학습을 위한 데이터 다운로드 (copied from H00br8GMGnT5)
path = kagglehub.dataset_download("asaniczka/mammals-image-classification-dataset-45-animals")


# 모델 학습을 위한 데이터 준비 (copied from 0eBp4cbiUJeO)
DATA_DIR = path + "/mammals"

image_paths = []
labels = []

for class_name in sorted(os.listdir(DATA_DIR)):
    class_dir = os.path.join(DATA_DIR, class_name)
    if os.path.isdir(class_dir):
        files = tf.io.gfile.glob(os.path.join(class_dir, "*.jpg"))
        image_paths.extend(files)
        labels.extend([class_name] * len(files))

class_names = sorted(list(set(labels)))
class_to_index = {name: idx for idx, name in enumerate(class_names)}
index_to_class = {idx: name for name, idx in class_to_index.items()}
numeric_labels = [class_to_index[label] for label in labels]

train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths,
    numeric_labels,
    test_size=0.2,
    stratify=numeric_labels,
    shuffle=True,
    random_state=42
)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths,
    temp_labels,
    test_size=0.5,
    stratify=temp_labels,
    shuffle=True,
    random_state=42
)

def path_label_to_dataset(paths, labels, batch_size=300, img_size=(256,256)):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def load_and_preprocess(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

    ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = path_label_to_dataset(train_paths, train_labels)
val_ds   = path_label_to_dataset(val_paths, val_labels)
test_ds  = path_label_to_dataset(test_paths, test_labels)


# 데이터 증강 설정
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"), # 좌우 반전
        layers.RandomRotation(0.1),      # 10% 범위 내에서 랜덤 회전
        layers.RandomZoom(0.1),          # 10% 범위 내에서 랜덤 줌
        layers.RandomTranslation(
            height_factor=0.1, width_factor=0.1
        ),                                      # 10% 범위 내에서 랜덤 이동
        layers.RandomContrast(0.2),    # 필요하다면 대비 조정 추가
    ],
    name="data_augmentation",
)

inputs = Input(shape=(256, 256, 3))
x = data_augmentation(inputs)

x = Conv2D(32, (3, 3), padding="same", activation='relu', name="conv1")(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
outputs = Dense(45, activation='softmax', name="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[early_stop, lr_scheduler])
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")

Using Colab cache for faster access to the 'mammals-image-classification-dataset-45-animals' dataset.


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Dense)                 │ (None, 45)             │    23,593,005 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,593,901 (90.00 MB)

 Trainable params: 23,593,901 (90.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 176ms/step - accuracy: 0.0392 - loss: 41.5771 - val_accuracy: 0.0633 - val_loss: 3.6811 - learning_rate: 0.0010
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.1261 - loss: 3.4405 - val_accuracy: 0.1855 - val_loss: 3.2141 - learning_rate: 0.0010
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.3881 - loss: 2.5106 - val_accuracy: 0.2233 - val_loss: 3.0234 - learning_rate: 0.0010
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.6467 - loss: 1.5828 - val_accuracy: 0.2553 - val_loss: 3.0131 - learning_rate: 0.0010
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.8442 - loss: 0.8689 - val_accuracy: 0.2356 - val_loss: 3.1550 - learning_rate: 0.0010
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - accuracy: 0.9394 - loss: 0.4382 - val_accuracy: 0.2269 - val_loss: 3.3678 - learning_rate: 0.0010
Epoch 7/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.9788 - loss: 0.2054


In [8]:
# 데이터 증강 설정
data_augmentation = tensorflow.keras.Sequential(
    [
        layers.RandomFlip("horizontal"), # 좌우 반전
        layers.RandomRotation(0.1),      # 10% 범위 내에서 랜덤 회전
        layers.RandomZoom(0.1),          # 10% 범위 내에서 랜덤 줌
        layers.RandomTranslation(
            height_factor=0.1, width_factor=0.1
        ),                                      # 10% 범위 내에서 랜덤 이동
        layers.RandomContrast(0.2),    # 필요하다면 대비 조정 추가
    ],
    name="data_augmentation",
)

inputs = Input(shape=(256, 256, 3))
x = data_augmentation(inputs)

# Conv Blocks
filter_list = [32, 64]

for filters in filter_list:
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
outputs = Dense(45, activation='softmax', name="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[early_stop, lr_scheduler])
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Dense)                 │ (None, 45)             │    11,796,525 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,815,917 (45.07 MB)

 Trainable params: 11,815,917 (45.07 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 152ms/step - accuracy: 0.0398 - loss: 6.8610 - val_accuracy: 0.1135 - val_loss: 3.3865 - learning_rate: 0.0010
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 147ms/step - accuracy: 0.1392 - loss: 3.3037 - val_accuracy: 0.1949 - val_loss: 3.0743 - learning_rate: 0.0010
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 147ms/step - accuracy: 0.1955 - loss: 3.0829 - val_accuracy: 0.2102 - val_loss: 3.0631 - learning_rate: 0.0010
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.2294 - loss: 2.9418 - val_accuracy: 0.2138 - val_loss: 3.1361 - learning_rate: 0.0010
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.2365 - loss: 2.8651 - val_accuracy: 0.1891 - val_loss: 3.4513 - learning_rate: 0.0010
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.2541 - loss: 2.8148 - val_accuracy: 0.2589 - val_loss: 2.8516 - learning_rate: 0.0010
Epoch 7/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.2524 - loss: 2.8055 - 

In [9]:
# 데이터 증강 설정2- 깊이2번 반복
data_augmentation = tensorflow.keras.Sequential(
    [
        layers.RandomFlip("horizontal"), # 좌우 반전
        layers.RandomRotation(0.1),      # 10% 범위 내에서 랜덤 회전
        layers.RandomZoom(0.1),          # 10% 범위 내에서 랜덤 줌
        layers.RandomTranslation(
            height_factor=0.1, width_factor=0.1
        ),                                      # 10% 범위 내에서 랜덤 이동
        layers.RandomContrast(0.2),    # 필요하다면 대비 조정 추가
    ],
    name="data_augmentation",
)

inputs = Input(shape=(256, 256, 3))
x = Rescaling(1./255)(inputs)
x = data_augmentation(inputs)

# Conv Blocks
filter_list = [32, 64, 128, 256]

for filters in filter_list:
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(45, activation='softmax', name="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[early_stop, lr_scheduler])
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 256)    │         1,02

 Total params: 17,992,269 (68.64 MB)

 Trainable params: 17,990,349 (68.63 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 34s 713ms/step - accuracy: 0.0241 - loss: 8.9372 - val_accuracy: 0.0240 - val_loss: 3.8063 - learning_rate: 0.0010
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0273 - loss: 3.8110 - val_accuracy: 0.0255 - val_loss: 3.9900 - learning_rate: 0.0010
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 695ms/step - accuracy: 0.0280 - loss: 3.8096 - val_accuracy: 0.0225 - val_loss: 4.2671 - learning_rate: 0.0010
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 666ms/step - accuracy: 0.0286 - loss: 3.8080
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0287 - loss: 3.8078 - val_accuracy: 0.0233 - val_loss: 9.8099 - learning_rate: 0.0010
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 693ms/step - accuracy: 0.0284 - loss: 3.7956 - val_accuracy: 0.0211 - val_loss: 7.2655 - learning_rate: 3.0000e-04
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 692ms/step - accuracy: 0.0285 - loss: 

In [10]:
# 데이터 증강 설정2- 깊이2번 반복 범위값수정
data_augmentation = tensorflow.keras.Sequential(
    [
        layers.RandomFlip("horizontal"), # 좌우 반전
        layers.RandomRotation(0.3),      # 30% 범위 내에서 랜덤 회전
        layers.RandomZoom(0.3),          # 30% 범위 내에서 랜덤 줌
        layers.RandomTranslation(
            height_factor=0.3, width_factor=0.3
        ),                                      # 30% 범위 내에서 랜덤 이동
        layers.RandomContrast(0.4),    # 필요하다면 대비 조정 추가
    ],
    name="data_augmentation",
)

inputs = Input(shape=(256, 256, 3))
x = Rescaling(1./255)(inputs)
x = data_augmentation(inputs)

# Conv Blocks
filter_list = [32, 64, 128, 256]

for filters in filter_list:
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(45, activation='softmax', name="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[early_stop, lr_scheduler])
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 32, 32, 256)    │         1,02

 Total params: 17,992,269 (68.64 MB)

 Trainable params: 17,990,349 (68.63 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 33s 709ms/step - accuracy: 0.0235 - loss: 9.3134 - val_accuracy: 0.0240 - val_loss: 3.8266 - learning_rate: 0.0010
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0216 - loss: 3.8128 - val_accuracy: 0.0073 - val_loss: 4.1440 - learning_rate: 0.0010
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0241 - loss: 3.8058 - val_accuracy: 0.0247 - val_loss: 7.7924 - learning_rate: 0.0010
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 668ms/step - accuracy: 0.0230 - loss: 3.8049
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 695ms/step - accuracy: 0.0230 - loss: 3.8049 - val_accuracy: 0.0255 - val_loss: 12.8064 - learning_rate: 0.0010
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 696ms/step - accuracy: 0.0242 - loss: 3.8046 - val_accuracy: 0.0247 - val_loss: 14.7602 - learning_rate: 3.0000e-04
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0250 - loss

In [11]:
# 데이터 증강 설정2- 깊이2번 반복 범위값수정
data_augmentation = tensorflow.keras.Sequential(
    [
        layers.RandomFlip("horizontal"), # 좌우 반전
        layers.RandomRotation(0.3),      # 30% 범위 내에서 랜덤 회전
        layers.RandomZoom(0.3),          # 30% 범위 내에서 랜덤 줌
        layers.RandomTranslation(
            height_factor=0.3, width_factor=0.3
        ),                                      # 30% 범위 내에서 랜덤 이동
        layers.RandomContrast(0.4),    # 필요하다면 대비 조정 추가
    ],
    name="data_augmentation",
)

inputs = Input(shape=(256, 256, 3))
x = Rescaling(1./255)(inputs)
x = data_augmentation(inputs)

# Conv Blocks
filter_list = [32, 64, 128, 256]

for filters in filter_list:
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(45, activation='softmax', name="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[early_stop, lr_scheduler])
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 32, 32, 256)    │         1,02

 Total params: 17,992,269 (68.64 MB)

 Trainable params: 17,990,349 (68.63 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 33s 709ms/step - accuracy: 0.0282 - loss: 13.4767 - val_accuracy: 0.0189 - val_loss: 4.9573 - learning_rate: 0.0010
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 696ms/step - accuracy: 0.0284 - loss: 4.6108 - val_accuracy: 0.0204 - val_loss: 4.1293 - learning_rate: 0.0010
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 695ms/step - accuracy: 0.0302 - loss: 4.1067 - val_accuracy: 0.0240 - val_loss: 4.2579 - learning_rate: 0.0010
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 697ms/step - accuracy: 0.0312 - loss: 3.8877 - val_accuracy: 0.0262 - val_loss: 3.8323 - learning_rate: 0.0010
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0365 - loss: 3.7772 - val_accuracy: 0.0291 - val_loss: 3.8654 - learning_rate: 0.0010
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 697ms/step - accuracy: 0.0437 - loss: 3.7636 - val_accuracy: 0.0298 - val_loss: 3.7965 - learning_rate: 0.0010
Epoch 7/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 694ms/step - accuracy: 0.0446 - loss: 3